In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [7]:
df.head()

,artist,song,text
0,John Waite,Run To Mexico,Come on baby don't you want to go \r\nI could...
1,Doors,Rock Me,Want you to rock me \r\nRock me all night lon...
2,Isley Brothers,If You Were There,You're the one that makes my day a dream come ...
3,Richard Marx,Playing With Fire,"Hey, you got the look of a lonely woman's eyes..."
4,Nina Simone,It Don't Mean A Thing,"Don't mean a thing, you can't swing \r\nDooa,..."


In [8]:
df['text'][0]

"Come on baby don't you want to go  \r\nI could take you there  \r\nYou could get what you want in the south  \r\nYou could let down your hair  \r\n  \r\nI said do you don't you will you won't you  \r\nBaby won't you please let me know  \r\nI ain't talkin' 'bout Chicago  \r\nI'm talkin' about Mexico  \r\n  \r\nNa na na na na  \r\nNa na na na na  \r\nMakin' a run for Mexico  \r\nMakin' a run for Mexico  \r\n  \r\nCould you  \r\nCome on baby cause I got to know  \r\nThe law is after me  \r\nI killed a man in a bar last night  \r\n  \r\nThere was no other way it could be  \r\nWhile the air in this joint could be cut with a knife  \r\nAs the jukebox got rotten selections  \r\nBring a compass and some money for gas  \r\n  \r\n'Cause I ain't gonna stop for directions  \r\nNot to Mexico  \r\nNa na na na na  \r\nMakin' a run to Mexico  \r\n  \r\nNa na na na na  \r\nMakin' a run to Mexico  \r\nCould you  \r\nOh oh  \r\n  \r\nCome on baby don't you want to go  \r\nI gotta get away  \r\nAnd every

In [9]:
df.shape

(5000, 3)

Text pre-processing

In [10]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True)

In [11]:
df.head()

,artist,song,text
0,John Waite,Run To Mexico,come on baby don't you want to go \r i could ...
1,Doors,Rock Me,want you to rock me \r rock me all night long...
2,Isley Brothers,If You Were There,you're the one that makes my day a dream come ...
3,Richard Marx,Playing With Fire,"hey, you got the look of a lonely woman's eyes..."
4,Nina Simone,It Don't Mean A Thing,"don't mean a thing, you can't swing \r dooa, ..."


In [12]:
import nltk
from nltk.stem.porter import PorterStemmer

In [24]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/jaydugad/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
stemmer = PorterStemmer()

In [14]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a =[stemmer.stem(w) for w in token]
    return " ".join(a)

In [15]:
token("You are beautiful")

'you are beauti'

In [16]:
df['text'].apply(lambda x: token(x))

0       come on babi do n't you want to go i could tak...
1       want you to rock me rock me all night long roc...
2       you 're the one that make my day a dream come ...
3       hey , you got the look of a lone woman 's eye ...
4       do n't mean a thing , you ca n't swing dooa , ...
                              ...                        
4995    my jesu , my savior lord , there is none like ...
4996    it 's been a long road , to get from there to ...
4997    you do n't rememb my name i do n't realli care...
4998    ( stephen bishop ) you call me from the room i...
4999    well come on pretti babi wo n't you walk with ...
Name: text, Length: 5000, dtype: object

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
tfid = TfidfVectorizer(analyzer='word', stop_words='english')

In [19]:
matrix = tfid.fit_transform(df['text'])

In [20]:
similar = cosine_similarity(matrix)

In [21]:
similar[0]

array([1.        , 0.0137754 , 0.01569978, ..., 0.00618666, 0.00330199,
       0.02102206])

In [25]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key = lambda x:x[1])
    song = []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [27]:
print(recommender("Rock Me"))

['Rock Bottom', "Rockin' Chair Money", "Rock 'n Roll High School", "Baby Let's Rock!"]


In [28]:
import pickle

In [31]:
pickle.dump(similar, open("similarity", "wb"))

In [32]:
pickle.dump(df, open("df", "wb"))